In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import AzureOpenAI
AzureOpenAI.api_key = os.getenv("AZURE_OPENAI_API_KEY")

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [2]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment = os.getenv("AZURE_OPENAI_LLM_MODEL"),
    api_version = "2025-01-01-preview",
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
)

In [3]:
llm.invoke([HumanMessage(content="Hello! I am Abhi, How are you?")])

AIMessage(content='Hello Abhi! I’m just a program, so I don’t have feelings, but I’m glad to be chatting with you. How about you—how are you doing today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 18, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-Cvg6637JJjOZNm4ZR4q1sIC1eTGFZ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_re

In [4]:
llm.invoke([
    HumanMessage(content="Hello! I am Abhi, How are you?"),
    AIMessage(content="Hello Abhi! I’m just a program, so I don’t have feelings, but I’m here and ready to chat with you. How are you doing today?"),
    HumanMessage(content="I am doing great! Can you tell me a joke?")
])

AIMessage(content='I’m glad to hear you’re doing great, Abhi! 😊  \nHere’s a joke for you:  \n\nWhy don’t skeletons fight each other?  \nBecause they don’t have the guts! 💀😂  \n\nWould you like another one?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 72, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-Cvg68XLY328i0WtXsD4PnCjieQ6rw', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {

In [5]:
## Message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    llm,
    get_session_history
)

def save_session_history(session_id: str, history: BaseChatMessageHistory) -> None:
    store[session_id] = history

In [8]:
config = {
    "configurable":{"session_id": "user_12345"},
}

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello! I am Abhi. I am an AI engineer.")],
    config=config
)
response.content

'Hey again, Abhi! Great to meet you. It’s awesome that you’re an AI engineer — always nice to talk with someone in the field. What kind of projects are you currently working on or most passionate about?'

In [12]:
config_1 = {
    "configurable":{"session_id": "user_12345"},
}

response = with_message_history.invoke(
    [HumanMessage(content="Whats' my name? What do I do?")],
    config=config_1
)

response.content

'Your name is **Abhi**, and you mentioned that you’re an **AI engineer**.'

In [13]:
## Using with different session id
config_2 = {
    "configurable":{"session_id": "user_67890"},
}

response = with_message_history.invoke(
    [HumanMessage(content="Whats' my name? What do I do?")],
    config=config_2
)

response.content

'I don’t have access to any personal information about you unless you share it with me in this chat. So I don’t know your name or what you do.  \n\nIf you tell me a bit about yourself—like your name, job, or interests—I can tailor my responses better!'

In [14]:
## Prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant that answers people's question to best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

In [15]:
chain = prompt | llm

In [16]:
chain.invoke({"messages": [
    HumanMessage(content="Hello! I am Abhi.")]})

AIMessage(content='Hello, Abhi! 👋  \nNice to meet you. How are you doing today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 33, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-CvgFfTo0nv4Swg6y5ulxZdZbR5D2g', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filt

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)

In [18]:
config = {"configurable":{"session_id": "user_10000"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello! I am Abhi. I am an AI engineer.")],
    config=config
)

response.content

'Hello, Abhi! It’s great to meet you. That’s awesome—you work as an AI engineer! What kind of projects or areas in AI are you currently working on or most interested in?'

In [21]:
## Add more complexity to the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant that answers people's question to best of your ability. Answer in the give language - {language}."),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | llm

In [22]:
chain.invoke({
    "messages": [
        HumanMessage(content="Hello! I am Abhi. I am an AI engineer."),
    ],
    "language": "French"
})

AIMessage(content='Bonjour Abhi\u202f! Ravi de faire ta connaissance. Êtes-vous spécialisé dans un domaine particulier de l’ingénierie de l’IA, comme l’apprentissage automatique, le traitement du langage naturel ou la vision par ordinateur\u202f?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 47, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-CvgQjM2tldXBxxoD4Pjrvf7bOpaJi', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': F

In [24]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id": "user_10001"}}
response = with_message_history.invoke(
    {
        "messages":[HumanMessage(content="Hello! I am Abhi. I am an AI engineer.")],
        "language":"Spanish"
    },
    config=config
)

response.content

'¡Hola, Abhi! Mucho gusto. Qué interesante que seas ingeniero en inteligencia artificial. ¿En qué área de la IA trabajas o qué tipo de proyectos te gusta desarrollar?'

In [25]:
response = with_message_history.invoke(
    {
        "messages":[HumanMessage(content="What's my name and profession?")],
        "language":"Spanish"
    },
    config=config
)

response.content

'Tu nombre es Abhi y tu profesión es ingeniero en inteligencia artificial.'

In [ ]:
## Manage the Conversaton History
